## Seldon V2 Kubernetes Examples


In [6]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

'172.18.255.9'

### Model

In [2]:
!cat ./models/sklearn-iris-gs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn
  memory: 100Ki


In [10]:
!kubectl create -f ./models/sklearn-iris-gs.yaml

model.mlops.seldon.io/iris created


In [11]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/iris condition met


In [12]:
!kubectl get model iris -n seldon-mesh -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-07-31T16:18:06Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2022-07-31T16:18:06Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [13]:
!seldon model infer iris --inference-host ${MESH_IP}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "052c263e-c5ba-4201-9c9e-d534e2612f92",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				2
			]
		}
	]
}


In [14]:
!seldon model infer iris --inference-mode grpc --inference-host ${MESH_IP}:80 \
   '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' | jq -M .

{
  "modelName": "iris_1",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [15]:
!kubectl get server mlserver -n seldon-mesh -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-07-31T16:17:23Z",
      "status": "True",
      "type": "Ready"
    },
    {
      "lastTransitionTime": "2022-07-31T16:17:23Z",
      "reason": "StatefulSet replicas matches desired replicas",
      "status": "True",
      "type": "StatefulSetReady"
    }
  ],
  "loadedModels": 1
}


In [16]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml

model.mlops.seldon.io "iris" deleted


### Experiment

In [27]:
!cat ./experiments/sklearn1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [28]:
!cat ./experiments/sklearn2.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [29]:
!kubectl create -f ./experiments/sklearn1.yaml
!kubectl create -f ./experiments/sklearn2.yaml

model.mlops.seldon.io/iris created
model.mlops.seldon.io/iris2 created


In [30]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/iris condition met
model.mlops.seldon.io/iris2 condition met


In [31]:
!cat ./experiments/ab-default-model.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: experiment-sample
  namespace: seldon-mesh
spec:
  default: iris
  candidates:
  - modelName: iris
    weight: 50
  - modelName: iris2
    weight: 50


In [32]:
!kubectl create -f ./experiments/ab-default-model.yaml 

experiment.mlops.seldon.io/experiment-sample created


In [33]:
!kubectl wait --for condition=ready --timeout=300s experiment --all -n seldon-mesh

experiment.mlops.seldon.io/experiment-sample condition met


In [34]:
!seldon model infer --inference-host ${MESH_IP}:80 -i 50 iris \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::26 :iris_1::24]


In [35]:
!kubectl delete -f ./experiments/ab-default-model.yaml 
!kubectl delete -f ./experiments/sklearn1.yaml
!kubectl delete -f ./experiments/sklearn2.yaml

experiment.mlops.seldon.io "experiment-sample" deleted
model.mlops.seldon.io "iris" deleted
model.mlops.seldon.io "iris2" deleted


### Pipeline - model chain

In [36]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki


In [37]:
!kubectl create -f ./models/tfsimple1.yaml
!kubectl create -f ./models/tfsimple2.yaml

model.mlops.seldon.io/tfsimple1 created
model.mlops.seldon.io/tfsimple2 created


In [38]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/tfsimple1 condition met
model.mlops.seldon.io/tfsimple2 condition met


In [39]:
!cat ./pipelines/tfsimples.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: tfsimples
  namespace: seldon-mesh
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
      inputs:
      - tfsimple1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple1.outputs.OUTPUT1: INPUT1
  output:
    steps:
    - tfsimple2


In [40]:
!kubectl create -f ./pipelines/tfsimples.yaml

pipeline.mlops.seldon.io/tfsimples created


In [41]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n seldon-mesh

pipeline.mlops.seldon.io/tfsimples condition met


In [42]:
!seldon pipeline infer tfsimples --inference-mode grpc --inference-host ${MESH_IP}:80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABYAAAAYAAAAGgAAABwAAAAeAAAAIAAAAA==",
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABY

In [43]:
!kubectl delete -f ./pipelines/tfsimples.yaml

pipeline.mlops.seldon.io "tfsimples" deleted


In [44]:
!kubectl delete -f ./models/tfsimple1.yaml
!kubectl delete -f ./models/tfsimple2.yaml

model.mlops.seldon.io "tfsimple1" deleted
model.mlops.seldon.io "tfsimple2" deleted


### Pipeline - model join

In [45]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml
!cat ./models/tfsimple3.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple3
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
  memory: 100Ki


In [46]:
!kubectl create -f ./models/tfsimple1.yaml
!kubectl create -f ./models/tfsimple2.yaml
!kubectl create -f ./models/tfsimple3.yaml

model.mlops.seldon.io/tfsimple1 created
model.mlops.seldon.io/tfsimple2 created
model.mlops.seldon.io/tfsimple3 created


In [47]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/tfsimple1 condition met
model.mlops.seldon.io/tfsimple2 condition met
model.mlops.seldon.io/tfsimple3 condition met


In [48]:
!cat ./pipelines/tfsimples-join.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: join
  namespace: seldon-mesh
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
    - name: tfsimple3      
      inputs:
      - tfsimple1.outputs.OUTPUT0
      - tfsimple2.outputs.OUTPUT1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple2.outputs.OUTPUT1: INPUT1
  output:
    steps:
    - tfsimple3


In [49]:
!kubectl create -f ./pipelines/tfsimples-join.yaml

pipeline.mlops.seldon.io/join created


In [50]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n seldon-mesh

pipeline.mlops.seldon.io/join condition met


In [52]:
!seldon pipeline infer join --inference-mode grpc --inference-host ${MESH_IP}:80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABYAAAAYAAAAGgAAABwAAAAeAAAAIAAAAA==",
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABY

In [53]:
!kubectl delete -f ./pipelines/tfsimples-join.yaml

pipeline.mlops.seldon.io "join" deleted


In [54]:
!kubectl delete -f ./models/tfsimple1.yaml
!kubectl delete -f ./models/tfsimple2.yaml
!kubectl delete -f ./models/tfsimple3.yaml

model.mlops.seldon.io "tfsimple1" deleted
model.mlops.seldon.io "tfsimple2" deleted
model.mlops.seldon.io "tfsimple3" deleted


## Explainer

In [85]:
!cat ./models/income.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/classifier"
  requirements:
  - sklearn


In [1]:
!kubectl create -f ./models/income.yaml

model.mlops.seldon.io/income created


In [2]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/income condition met


In [3]:
!kubectl get model income -n seldon-mesh -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-07-31T16:56:50Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2022-07-31T16:56:50Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [7]:
!seldon model infer income --inference-host ${MESH_IP}:80 \
     '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income_1",
	"model_version": "1",
	"id": "4855f9f7-83b5-4ed3-9d95-4b046433ae77",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				0
			]
		}
	]
}


In [8]:
!cat ./models/income-explainer.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-explainer
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/scv2/examples/income/explainer"
  explainer:
    type: anchor_tabular
    modelRef: income


In [9]:
!kubectl create -f ./models/income-explainer.yaml

model.mlops.seldon.io/income-explainer created


In [10]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/income condition met
model.mlops.seldon.io/income-explainer condition met


In [11]:
!kubectl get model income-explainer -n seldon-mesh -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-07-31T16:57:21Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2022-07-31T16:57:21Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [12]:
!seldon model infer income-explainer --inference-host ${MESH_IP}:80 \
     '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income-explainer_1",
	"model_version": "1",
	"id": "a69024db-711d-4592-a17c-082ec490071a",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "explanation",
			"shape": [
				1
			],
			"datatype": "BYTES",
			"parameters": {
				"content_type": "str",
				"headers": null
			},
			"data": [
				"{\"meta\": {\"name\": \"AnchorTabular\", \"type\": [\"blackbox\"], \"explanations\": [\"local\"], \"params\": {\"seed\": 1, \"disc_perc\": [25, 50, 75], \"threshold\": 0.95, \"delta\": 0.1, \"tau\": 0.15, \"batch_size\": 100, \"coverage_samples\": 10000, \"beam_size\": 1, \"stop_on_first\": false, \"max_anchor_size\": null, \"min_samples_start\": 100, \"n_covered_ex\": 10, \"binary_cache_size\": 10000, \"cache_margin\": 1000, \"verbose\": false, \"verbose_every\": 1, \"kwargs\": {}}, \"version\": \"0.7.0\"}, \"data\": {\"anchor\": [\"Marital Status = Never-Married\", \"Relationship = Own-child\"], \"precision\": 0.9578947368421052, \"co

In [13]:
!kubectl delete -f ./models/income.yaml
!kubectl delete -f ./models/income-explainer.yaml

model.mlops.seldon.io "income" deleted
model.mlops.seldon.io "income-explainer" deleted


## Custom Server

In [68]:
!cat ./servers/custom-mlserver.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Server
metadata:
  name: mlserver-custom
  namespace: seldon-mesh
spec:
  serverConfig: mlserver
  podSpec:
    containers:
    - image: cliveseldon/mlserver:1.1.0.explain
      name: mlserver


In [69]:
!kubectl create -f ./servers/custom-mlserver.yaml

server.mlops.seldon.io/mlserver-custom created


In [70]:
!kubectl wait --for condition=ready --timeout=300s server --all -n seldon-mesh

server.mlops.seldon.io/mlserver condition met
server.mlops.seldon.io/triton condition met
error: timed out waiting for the condition on servers/mlserver-custom


In [71]:
!cat ./models/iris-custom-server.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  server: mlserver-custom


In [72]:
!kubectl create -f ./models/iris-custom-server.yaml

model.mlops.seldon.io/iris created


In [73]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/iris condition met


In [74]:
!seldon model infer iris --inference-host ${MESH_IP}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "f0102b20-c0a9-46ea-8144-8a5164134465",
	"parameters": {
		"content_type": null,
		"headers": null
	},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				2
			]
		}
	]
}


In [75]:
!kubectl delete -f ./models/iris-custom-server.yaml

model.mlops.seldon.io "iris" deleted


In [76]:
!kubectl delete -f ./servers/custom-mlserver.yaml

server.mlops.seldon.io "mlserver-custom" deleted
